### 검색도구: Tavily Search

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

### Agent 가 사용할 도구 목록 정의

In [4]:
# tools 리스트에 search 도구를 추가합니다.
tools = [search]

### Agent with smaller model

In [9]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage

from langchain.pydantic_v1 import BaseModel, Field


local_llm = ChatOpenAI(
    base_url="http://localhost:3000",
    api_key="lm-studio",
    model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
    temperature=0,
)

tools = [search]

llm_with_tools = local_llm.bind_tools(tools)

In [10]:
from typing import Tuple, List

chat_model_with_stop = local_llm.bind(stop=["Observation", "\nObservation", "\n관측"])

# Inspiration taken from hub.pull("hwchase17/react-json")
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [11]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
        "chat_history": lambda x: (
            _format_chat_history(x["chat_history"]) if x.get(
                "chat_history") else []
        ),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
).with_types(input_type=AgentInput)

In [13]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {
        "input": "판교 제로원에이아이 전화번호를 웹 검색하여 결과를 알려주세요.",
        "chat_history": [],
    }
)
print(f'답변: {response["output"]}')
# API 연결 문제로 실행 실패

In [14]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolExecutor
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnableParallel


def format_search_result(search_result):
    return " ".join([r["content"] for r in search_result])


tool_executor = ToolExecutor(tools)

In [15]:
search_agent = (
    agent_executor
    | itemgetter("output")
    | ReActJsonSingleInputOutputParser()
    | tool_executor
    | format_search_result
)

search_agent_parallel = RunnableParallel(
    context=search_agent, question=itemgetter("input")
)

In [18]:
search_agent_parallel.invoke(
    {
        "input": "판교 제로원에이아이 전화번호 웹 검색하여 알려줘.",
        "chat_history": [],
    }
)
# API 연결문제로 실행 실패

In [19]:
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Assistant. You must answer the question based on the context.\n#Context: {context}",
        ),
        ("user", "{question}"),
    ]
)

eeve = ChatOllama(
    model="EEVE-Korean-10.8B:long",
    temperature=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

chain = (
    RunnableParallel(question=itemgetter("question"),
                     context=itemgetter("context"))
    | prompt
    | eeve
    | StrOutputParser()
)

In [20]:
final_chain = search_agent_parallel | chain

In [22]:
final_chain.invoke(
    {
        "input": "판교 제로원에이아이 전화번호 검색하여 알려주세요",
        "chat_history": [],
    }
)
# API 연결문제로 실행 실패